# Exploring the UTx000 Extension Beacon Data
(Known as BPEACE2 in the [GH repo](https://github.com/intelligent-environments-lab/utx000))

In [1]:
import warnings
warnings.filterwarnings('ignore')

# Evening IEQ Measurements
Now that we have parsed the beacon data into time periods corresponding to when participants were asleep, it would be interesting to analyze what concentrations are being experienced during the evening.

## Package Import

In [2]:
import os
import sys
sys.path.append('../')

from src.features import build_features
from src.visualization import visualize
from src.reports import make_report

import pandas as pd
import numpy as np

from datetime import datetime, timedelta

import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates
from matplotlib.colors import ListedColormap, LinearSegmentedColormap

# Data Import

## Beacon Data
We are only concerned with the beacon data when participants are asleep since that is the only time we can be sure participants are home in their bedrooms.

In [3]:
beacon = pd.read_csv('../data/processed/bpeace2-beacon-fb_and_gps_restricted.csv',
                    index_col=0,parse_dates=[0,-1,-2],infer_datetime_format=True)
beacon.head()

,Lat,Long,Alt,Accuracy,TVOC,eCO2,Lux,Visible,Infrared,NO2,...,PM_C_2p5,PM_C_4,PM_C_10,z,Beacon,Beiwe,Fitbit,REDCap,start_time,end_time
2020-08-10 05:10:00,30.58838,-97.42591,113.98145,5.29684,185.50,750.25,2.04,1.0,0.0,288.95,...,6.748117,10.714056,11.507244,1.441551,21,lkkjddam,25,12,2020-08-10 04:42:30,2020-08-10 12:35:30
2020-08-10 05:15:00,NaN,NaN,NaN,NaN,166.08,706.48,2.04,1.0,0.0,277.12,...,6.515844,10.288899,11.043510,1.468756,21,lkkjddam,25,12,2020-08-10 04:42:30,2020-08-10 12:35:30
2020-08-10 05:20:00,NaN,NaN,NaN,NaN,151.40,666.48,2.04,1.0,0.0,277.44,...,6.215933,9.795078,10.510907,1.492578,21,lkkjddam,25,12,2020-08-10 04:42:30,2020-08-10 12:35:30
2020-08-10 05:25:00,NaN,NaN,NaN,NaN,158.84,686.92,2.04,1.0,0.0,282.44,...,6.486098,10.357869,11.132223,1.457762,21,lkkjddam,25,12,2020-08-10 04:42:30,2020-08-10 12:35:30
2020-08-10 05:30:00,NaN,NaN,NaN,NaN,146.15,669.20,2.04,1.0,0.0,272.80,...,6.721101,10.739605,11.543306,1.430890,21,lkkjddam,25,12,2020-08-10 04:42:30,2020-08-10 12:35:30


## Participant Information
We need the participant beacon, beiwe, and fitbit IDs so we can cross-reference the various pieces of information.

In [18]:
# getting pt names
names = pd.read_excel('../data/raw/bpeace2/admin/id_crossover.xlsx',sheet_name='all')
names = names[['Beiwe ID','First','Last']]
# getting beacon only pts with names
info = pd.read_excel('../data/raw/bpeace2/admin/id_crossover.xlsx',sheet_name='beacon')
info = info[['REDCap','Beiwe','Beacon','Lat','Long','Volume','Roommates']] # keep their address locations
info = info.merge(left_on='Beiwe',right=names,right_on='Beiwe ID')
info.head()

,REDCap,Beiwe,Beacon,Lat,Long,Volume,Roommates,Beiwe ID,First,Last
0,0,zdpffrox,7,30.326815,-97.744665,1365,1,zdpffrox,Hagen,Fritz
1,4,idbkjh8u,30,30.284625,-97.749419,1080,0,idbkjh8u,Alana,Martinez
2,10,kyj367pi,1,30.312614,-97.732338,1080,0,kyj367pi,Angela,Gordillo
3,12,lkkjddam,21,30.588384,-97.428051,1188,0,lkkjddam,Austin,Reynoso
4,15,rnse61g4,34,30.290757,-97.745684,1080,0,rnse61g4,Chloe,Roman


## Fitbit Data
We will need some information from each of the participants that Fitbit logs like body weight.

In [19]:
daily = pd.read_csv('../data/processed/bpeace2-fitbit-daily.csv',
                   index_col=0,parse_dates=True,infer_datetime_format=True)
daily.head()

,activities_calories,activities_caloriesBMR,activities_steps,activities_distance,activities_minutesSedentary,activities_minutesLightlyActive,activities_minutesFairlyActive,activities_minutesVeryActive,activities_activityCalories,body_bmi,body_fat,body_weight,foods_log_caloriesIn,foods_log_water,beiwe
date,,,,,,,,,,,,,,,
2020-05-13,2781.0,1876.0,9207,4.396294,1241,70,118,11,1097.0,23.754,0.0,180.0,0.0,0.0,hfttkth7
2020-05-14,3727.0,1876.0,15207,7.261114,614,263,134,23,2234.0,23.754,0.0,180.0,0.0,0.0,hfttkth7
2020-05-15,3909.0,1876.0,14556,8.028501,577,205,57,108,2381.0,23.754,0.0,180.0,0.0,0.0,hfttkth7
2020-05-16,3927.0,1876.0,18453,8.748670,760,176,24,151,2364.0,23.754,0.0,180.0,0.0,0.0,hfttkth7
2020-05-17,4180.0,1876.0,15425,7.973149,605,207,50,131,2652.0,23.754,0.0,180.0,0.0,0.0,hfttkth7


# Ventilation based on constant CO2 concentration
We can estimate the ventilation rate of the participants' bedrooms based on a constant CO2 concentration:

$$
\lambda = \frac{E}{V \left(C - pC_0\right)}
$$

In our case we have to make quite a few assumptions:
- $E$: Emission rate of CO2 can be estimated for humans to be approximately $30 g/hr$. We can further refine this value by considering the height, weight, sex, and activity level of the participant - all information that we have access to. 
- $V$: Arguably the most difficult value to track down and the one that could affect the answer the most. We can assume a volume of each of the participants bedrooms, but there is the possiblility of refining this value by cross-referencing the participants' addresses with any known apartment complexes. From there, we can estimate the floor plan based on the number of roommates.
- $p$: Penetration factor of CO2 from outside into the indoor space - we can safely assume this to be 1.
- $C_0$: Outdoor concentration of CO2 - another safe assumption of 400 ppm. 

## Emission Rate
Based on an article that Sangeetha sent me, we can determine the emission rate of CO2 from the following equation:

$$
V_{CO_2} = (BMR)(M)(T/P)(0.000179)
$$

where:
- $V_{CO_2}$ is the volumetric emission rate of CO2 in L/s
- $BMR$ is the body mass ratio determined from Persily and De Jonge 2016
- $M$: Activity level in METs
- $T$ is the temperature in Kelvin
- $P$ is the pressure in kPa

### Body Mass Ratio
The body mass ratio is given by one of the two equations since our participants are in the age range of 18-30

$$
BMR_{male} = 0.063m + 2.896\\
BMR_{female} = 0.062m + 2.036
$$

where $m$ is the body mass in kilograms. We have the pariticpants body weights assuming they logged them into Fitbit. We can grab these data and put it alongside the participants beacon, beiwe, and fitbit IDs.

We will take the body weight for each participant, average it over the total amount of time logged (just in case they changed it which I doubt anyone did) and store that value in our ```ventilation_info``` dataframe.

In [20]:
weight_dict = {'Beiwe':[],'Mass':[]}
for pt in daily['beiwe'].unique():
    daily_pt = daily[daily['beiwe'] == pt]
    weight_dict['Beiwe'].append(pt)
    weight_dict['Mass'].append(np.nanmean(daily_pt['body_weight'])*0.453592) # convert to kg
    
mass_info = info.merge(left_on='Beiwe',right=pd.DataFrame(weight_dict),right_on='Beiwe')
mass_info.head()

,REDCap,Beiwe,Beacon,Lat,Long,Volume,Roommates,Beiwe ID,First,Last,Mass
0,0,zdpffrox,7,30.326815,-97.744665,1365,1,zdpffrox,Hagen,Fritz,74.842680
1,4,idbkjh8u,30,30.284625,-97.749419,1080,0,idbkjh8u,Alana,Martinez,72.574720
2,10,kyj367pi,1,30.312614,-97.732338,1080,0,kyj367pi,Angela,Gordillo,52.014033
3,12,lkkjddam,21,30.588384,-97.428051,1188,0,lkkjddam,Austin,Reynoso,75.388768
4,15,rnse61g4,34,30.290757,-97.745684,1080,0,rnse61g4,Chloe,Roman,54.431040


In order to properly calculate the BMR of the participants, we need to know whether they are male or female. Participants had to mark whether or not they were male or female on the CRISIS baseline survey we sent via REDCap. 

In [30]:
# getting sex from baseline survey
crisis_baseline = pd.read_csv('../data/raw/bpeace2/surveys/CRISIS_E3_labels.csv',
                              usecols=[0,3],names=['REDCap','Sex'],header=0)
sex_info = mass_info.merge(left_on='REDCap',right=crisis_baseline,right_on='REDCap',how='outer')
sex_info.dropna(subset=['Beacon'],axis=0,inplace=True)
# adding sex manually
sex_info.iloc[0,-1] = 'Male' # Hagen Fritz
sex_info.iloc[15,-1] = 'Female' # Katherine Mai
sex_info.iloc[16,-1] = 'Female' # Elizabeth Avalos
sex_info.head()

,REDCap,Beiwe,Beacon,Lat,Long,Volume,Roommates,Beiwe ID,First,Last,Mass,Sex
0,0,zdpffrox,7.0,30.326815,-97.744665,1365.0,1.0,zdpffrox,Hagen,Fritz,74.842680,Male
1,4,idbkjh8u,30.0,30.284625,-97.749419,1080.0,0.0,idbkjh8u,Alana,Martinez,72.574720,Female
2,10,kyj367pi,1.0,30.312614,-97.732338,1080.0,0.0,kyj367pi,Angela,Gordillo,52.014033,Female
3,12,lkkjddam,21.0,30.588384,-97.428051,1188.0,0.0,lkkjddam,Austin,Reynoso,75.388768,Male
4,15,rnse61g4,34.0,30.290757,-97.745684,1080.0,0.0,rnse61g4,Chloe,Roman,54.431040,Female


Now we calculate the BMR based on the mass and sex of the participants

In [31]:
def get_BMR(sex, mass):
    '''
    Calculates the BMR based on mass assuming an age range between 18 and 30
    
    Inputs:
    - sex: string in ['Male','Female']
    - mass: double of the mass in kg
    
    Returns BMR from Persily and De Jong 2016
    '''
    if sex.lower() == 'male':
        return 0.063*mass + 2.896
    elif sex.lower() == 'female':
        return 0.062*mass + 2.036
    else:
        return 0

In [35]:
sex_info['BMR'] = sex_info.apply(lambda row: get_BMR(row['Sex'],row['Mass']),axis=1)
pt_info = sex_info.set_index('Beiwe')
pt_info.head()

,REDCap,Beacon,Lat,Long,Volume,Roommates,Beiwe ID,First,Last,Mass,Sex,BMR
Beiwe,,,,,,,,,,,,
zdpffrox,0,7.0,30.326815,-97.744665,1365.0,1.0,zdpffrox,Hagen,Fritz,74.842680,Male,7.611089
idbkjh8u,4,30.0,30.284625,-97.749419,1080.0,0.0,idbkjh8u,Alana,Martinez,72.574720,Female,6.535633
kyj367pi,10,1.0,30.312614,-97.732338,1080.0,0.0,kyj367pi,Angela,Gordillo,52.014033,Female,5.260870
lkkjddam,12,21.0,30.588384,-97.428051,1188.0,0.0,lkkjddam,Austin,Reynoso,75.388768,Male,7.645492
rnse61g4,15,34.0,30.290757,-97.745684,1080.0,0.0,rnse61g4,Chloe,Roman,54.431040,Female,5.410724


### Activity Level in METs
The Persily and De Jong 2016 article gives values for activity level in METs dependingon the activity. We can assume that our participants are asleep since we are only using data when Fitbit detects sleeping individuals. The MET value for a sleeping individual, from the article, is **0.95**.

### Temperature
The **temperature will vary depending on the night** that we choose to analyze and therefore will alter the emission rate of CO2. Thus we cannot establish a static emission rate.

### Pressure
The pressure will more or less remain constant since we are at the same altitude for the most part in Austin. The National Weather Service has a [station at the airport](https://w1.weather.gov/data/obhistory/KAUS.html) in Austin that reports the atmopsheric pressure is around **102.5 kPa**.

### Python Function
Now that we have all this information, we can create the Python function to get the emission rate for each participant.

In [38]:
def get_emission_rate(BMR, T):
    '''
    Calculates the CO2 emission rate
    
    Inputs:
    - BMR: double of body-mass-ratio
    - T: double of Temperature in Kelvin
    
    Returns CO2 emission rate in L/s
    '''
    
    # Assumed constants
    M = 0.95 #METs
    P = 102.5 #kPa
    
    return BMR * M * (T / P) * 0.000179

## Volume
We can always assume a value for the volume of the space, or we can go a bit more in-depth by looking at the addresses of the participants.

Doing this presents their own set of problems since we now have to trust the answers of the participants and the addresses could be different than the addresess we sent the beacons to. Rather, we can simply use average numbers:
- **Stand-Alone Home**: We can use the US average of 11x12 feet with 9 foot ceilings - **1188 feet**
- **Apartment**: Another average value would be 10x12 with 9 foot ceilings - **1080 feet**

<div class="alert-block alert alert-success">
The volume values have already been added to the info.csv file for each participant based on the address used to ship the beacons to, <b>not</b> the response in the EE survey.
<div>

## Outdoor CO2
We can assume the outdoor CO2 concentration is 400 ppm, but we need to convert this value into g/m$^3$.

In [41]:
def convert_ppm_to_gm3(concentration, mm=44.0, mv=24.5):
    '''
    Converts the ppm of a gas to g/m3
    
    Inputs:
    - concentration: double specifying the concentration in ppm
    - mm: double specifying the molar mass of the compound (default is CO2)
    - mv: double of molar volume (default is at room temperature)
    
    Returns concentration in g/m3
    '''
    
    return concentration / 10**6 * mm / mv * 1000

**Testing**

Compare to calculator [here](https://www.teesing.com/en/page/library/tools/ppm-mg3-converter)

In [43]:
convert_ppm_to_gm3(400)

0.7183673469387756

## Constant CO2
Now we need to find periods during the evening when the CO2 concentration is constant. We can do this by looking at the mean difference in CO2 concentration during a certain window. If the change is less than a certain threshold, we can then assume the concentration is stable.

We can also cross-reference with the temperature to make sure that the AC isn't jsut cycling on and off quickly keeping the CO2 within a tight window. 

### Processing Temperature
We want to make sure to use the temperature sensors from the DGS sensors rather than the CO2 sensor. There might be times when one of the temperature sensors is working or they are both operating. In the latter case, we want to average the values and then combine the temperature values into one column.

We will also need the temperature to better estimate the emission rate.

In [10]:
beacon['T'] = beacon[['T_CO','T_NO2']].mean(axis=1)
beacon['RH'] = beacon[['RH_CO','RH_NO2']].mean(axis=1)
beacon_co2 = beacon[['REDCap','Beiwe','Beacon','Fitbit','CO2','T','RH','start_time','end_time']]
beacon_co2.head()

,REDCap,Beiwe,Beacon,Fitbit,CO2,T,RH,start_time,end_time
2020-08-10 05:10:00,12,lkkjddam,21,25,1342.427700,24.0,33.15,2020-08-10 04:42:30,2020-08-10 12:35:30
2020-08-10 05:15:00,12,lkkjddam,21,25,1326.702582,24.0,33.00,2020-08-10 04:42:30,2020-08-10 12:35:30
2020-08-10 05:20:00,12,lkkjddam,21,25,1300.445127,24.0,33.00,2020-08-10 04:42:30,2020-08-10 12:35:30
2020-08-10 05:25:00,12,lkkjddam,21,25,1262.876597,24.0,33.04,2020-08-10 04:42:30,2020-08-10 12:35:30
2020-08-10 05:30:00,12,lkkjddam,21,25,1301.729658,24.0,33.00,2020-08-10 04:42:30,2020-08-10 12:35:30


### Constant CO2 Function
The function below gets the periods of constant CO2 - read the function help text for more information

In [12]:
def get_constant_co2_periods(df,window_size=12,co2_threshold=10,t_threshold=0.25):
    '''
    Gets the periods of constant CO2 for the specified window size
    
    Inputs:
    - df: dataframe holding CO2 and T columns indexed by timestamp
    - window_size: interger specifying how many 5-min time steps to look for constant CO2/T
    - co2_threshold: integer specifying how tight of a window to restrict the variation of CO2 to
    - t_threshold: float specifying how tight of a window to restrict the variation of T to
    
    Returns a dictionary with dataframes of the CO2 and T indexed by time with keys correspond to the period
    '''
    
    period_dict = {}
    df = df.resample('5T').mean()
    df.fillna(0,inplace=True)
    
    df['dCO2'] = abs(df['CO2'] - df['CO2'].shift(1))
    df['dCO2_window'] = df['dCO2'].rolling(window_size,closed='left').mean()
    df['dCO2_window'] = df['dCO2_window'].shift(-1*(window_size-1))
    
    df['dT'] = abs(df['T'] - df['T'].shift(1))
    df['dT_window'] = df['dT'].rolling(window_size,closed='left').mean()
    df['dT_window'] = df['dT_window'].shift(-1*(window_size-1))
    
    period = 1
    for i in range(len(df)):
        if df['dCO2_window'][i] < co2_threshold and df['dT_window'][i] < t_threshold:
            period_dict[period] = df[i:i+window_size]
            period += 1
    
    return period_dict

## Air Exchange Rate
Now we have all the information we need for the equation, we can determine the air exchange rate in each of the participants' homes.

We do need to find some extra information to convert our units:
- **Density of CO2 ($\rho_{CO_2}$)**: The density of CO2 can be found [here](from www.engineeringtoolbox.com/carbon-dioxide-density-specific-weight-temperature-pressure-d_2018.html). We want to look for nearly atmospheric pressure and room temperature.
- **Conversions**: Like cubic feet to cubic meters, ppm to g/m$^3$, and the like.

In [44]:
def get_ach_from_constant_co2(E, V, C, C0=400.0, p=1.0):
    '''
    Calculates the air exchange rate for constant CO2 events
    
    Inputs:
    - E: double of emission rate in L/s
    - V: double of volume in ft3
    - C: double of room co2 concetration in ppm
    - C0: double of outdoor co2 concentration in ppm (default is 400)
    - p: double of penetration factor (default is 1)
    
    Returns ach in 1/h
    '''
    # defining constants
    rho = 1.8 # g/L 
    
    # converting units
    E_gs = E * rho # L/s to g/s
    V_m3 = V * 0.0283168 # ft3 to m3
    C_gm3 = convert_ppm_to_gm3(C) # ppm to g/m3
    C0_gm3 = convert_ppm_to_gm3(C0) # ppm to g/m3
    
    return E_gs / (V_m3 * (C_gm3 - p*C0_gm3)) * 3600

## Calculating emission rates and ventilation rates
Putting everything together now we loop through each night for each participant, calculate the emission rate, and then determine the ventilation rate.

In [82]:
info_pt

,REDCap,Beacon,Lat,Long,Volume,Roommates,Beiwe ID,First,Last,Mass,Sex,BMR
Beiwe,,,,,,,,,,,,
lkkjddam,12,21.0,30.588384,-97.428051,1188.0,0.0,lkkjddam,Austin,Reynoso,75.388768,Male,7.645492


In [83]:
ventilation_df = pd.DataFrame()
for pt in beacon_co2['Beiwe'].unique(): # cycling through each of the participants
    # setting up the dictionary to add pt values to
    pt_dict = {'Beiwe':[],'Beacon':[],'Start':[],'End':[],'Avg_CO2':[],'dCO2':[],'Avg_T':[],'dT':[],'E':[],'ACH':[]}
    # pt-specific dataframes
    beacon_pt = beacon_co2[beacon_co2['Beiwe'] == pt]
    info_pt = pt_info[pt_info.index == pt]
    
    for start, end in zip(beacon_pt['start_time'].unique(),beacon_pt['end_time'].unique()): # looping through sleep events
        beacon_pt_night = beacon_pt[start:end] # masking for iaq data during sleep
        if len(beacon_pt_night) > 36: # looking for nights when we have at least 3 hours (beacon data is at 5-min)
            constant_periods = get_constant_co2_periods(beacon_pt_night[['CO2','T','RH']],window_size=24)
            for period in constant_periods.keys():
                C = np.mean(constant_periods[period]['CO2'])
                if C > 400: # want to make sure we are including viable nights
                    pt_dict['Beiwe'].append(pt)
                    pt_dict['Beacon'].append(info_pt['Beacon'].values[0])
                    pt_dict['Start'].append(start)
                    pt_dict['End'].append(end)
                    
                    T = np.mean(constant_periods[period]['T'])
                    pt_dict['Avg_T'].append(T)
                    pt_dict['dT'].append(np.mean(abs(constant_periods[period]['T'] - constant_periods[period]['T'].shift(1))))
                    
                    pt_dict['Avg_CO2'].append(C)
                    pt_dict['dCO2'].append(np.mean(abs(constant_periods[period]['CO2'] - constant_periods[period]['CO2'].shift(1))))
                    
                    E = get_emission_rate(pt_info.loc[pt,'BMR'],T+273)
                    pt_dict['E'].append(E)
                    
                    V = info_pt['Volume'].values[0]
                    ACH = get_ach_from_constant_co2(E,V,C)
                    pt_dict['ACH'].append(ACH)
                
    ventilation_df = ventilation_df.append(pd.DataFrame(pt_dict))
                

In [85]:
ventilation_df

,Beiwe,Beacon,Start,End,Avg_CO2,dCO2,Avg_T,dT,E,ACH
0,lkkjddam,21.0,2020-08-21 03:02:30,2020-08-21 11:30:00,1078.040512,8.941249,26.500000,0.130435,0.003799,0.600935
1,lkkjddam,21.0,2020-08-21 03:02:30,2020-08-21 11:30:00,1079.606943,8.686468,26.541667,0.130435,0.003799,0.599633
2,lkkjddam,21.0,2020-08-21 03:02:30,2020-08-21 11:30:00,1080.794088,8.417760,26.583333,0.130435,0.003800,0.598671
3,lkkjddam,21.0,2020-08-21 03:02:30,2020-08-21 11:30:00,1081.979480,8.605416,26.625000,0.130435,0.003800,0.597713
4,lkkjddam,21.0,2020-08-21 03:02:30,2020-08-21 11:30:00,1083.629814,8.775572,26.666667,0.108696,0.003801,0.596353
...,...,...,...,...,...,...,...,...,...,...
32,9xmhtq74,17.0,2020-07-01 22:41:30,2020-07-02 06:02:30,1127.403130,7.734254,28.786667,0.106957,0.003061,0.496488
33,9xmhtq74,17.0,2020-07-01 22:41:30,2020-07-02 06:02:30,1126.191126,7.768879,28.745000,0.121739,0.003061,0.497248
34,9xmhtq74,17.0,2020-07-01 22:41:30,2020-07-02 06:02:30,1125.372044,7.562292,28.703333,0.121739,0.003060,0.497741
35,9xmhtq74,17.0,2020-07-01 22:41:30,2020-07-02 06:02:30,1124.253177,7.643068,28.661667,0.121739,0.003060,0.498441


In [86]:
ventilation_df = pd.DataFrame()
for pt in beacon_co2['Beiwe'].unique(): # cycling through each of the participants
    # setting up the dictionary to add pt values to
    pt_dict = {'Beiwe':[],'Beacon':[],'Start':[],'End':[],'Avg_CO2':[],'dCO2':[],'Avg_T':[],'dT':[],'E':[],'ACH':[]}
    # pt-specific dataframes
    beacon_pt = beacon_co2[beacon_co2['Beiwe'] == pt]
    info_pt = pt_info[pt_info.index == pt]
    
    for start, end in zip(beacon_pt['start_time'].unique(),beacon_pt['end_time'].unique()): # looping through sleep events
        beacon_pt_night = beacon_pt[start:end] # masking for iaq data during sleep
        if len(beacon_pt_night) > 36: # looking for nights when we have at least 3 hours (beacon data is at 5-min)
            constant_periods = get_constant_co2_periods(beacon_pt_night[['CO2','T','RH']],window_size=24)
            n = len(constant_periods)
            if n > 0:
                C = 0
                dC = 0
                T = 0
                dT = 0
                for period in constant_periods.keys():
                    C += np.mean(constant_periods[period]['CO2'])
                    dC += np.nanmean(abs(constant_periods[period]['CO2'] - constant_periods[period]['CO2'].shift(1)))

                    T += np.mean(constant_periods[period]['T'])
                    dT += np.nanmean(abs(constant_periods[period]['T'] - constant_periods[period]['T'].shift(1)))
                
                C /= n
                dC /= n
                T /= n
                dT /= n
                if C > 400:
                    pt_dict['Beiwe'].append(pt)
                    pt_dict['Beacon'].append(info_pt['Beacon'].values[0])
                    pt_dict['Start'].append(start)
                    pt_dict['End'].append(end)
                    
                    pt_dict['Avg_CO2'].append(C)
                    pt_dict['dCO2'].append(dC)

                    pt_dict['Avg_T'].append(T)
                    pt_dict['dT'].append(dT)

                    E = get_emission_rate(pt_info.loc[pt,'BMR'],T+273)
                    pt_dict['E'].append(E)

                    V = info_pt['Volume'].values[0]
                    ACH = get_ach_from_constant_co2(E,V,C)
                    pt_dict['ACH'].append(ACH)
        
    ventilation_df = ventilation_df.append(pd.DataFrame(pt_dict))
                

In [87]:
ventilation_df

,Beiwe,Beacon,Start,End,Avg_CO2,dCO2,Avg_T,dT,E,ACH
0,lkkjddam,21.0,2020-08-21 03:02:30,2020-08-21 11:30:00,1091.050046,8.175281,26.686667,0.100909,0.003801,0.589989
0,745vq78e,5.0,2020-06-15 03:45:00,2020-06-15 10:02:30,1180.402548,6.304758,28.000000,0.000000,0.002674,0.367490
1,745vq78e,5.0,2020-06-16 01:59:00,2020-06-16 09:55:00,1116.553173,5.989771,28.448545,0.057159,0.002678,0.400832
2,745vq78e,5.0,2020-06-19 05:00:00,2020-06-19 12:27:30,1240.341093,8.297229,28.109787,0.027466,0.002675,0.341403
3,745vq78e,5.0,2020-06-20 05:32:00,2020-06-20 16:28:00,1330.425592,6.310843,28.047295,0.028392,0.002674,0.308284
4,745vq78e,5.0,2020-06-21 05:31:00,2020-06-21 11:11:30,1014.939014,5.301613,27.995020,0.007526,0.002674,0.466364
5,745vq78e,5.0,2020-06-26 05:57:30,2020-06-26 11:45:30,1174.297188,5.571016,28.022390,0.009634,0.002674,0.370415
6,745vq78e,5.0,2020-06-28 06:13:00,2020-06-28 14:09:00,1308.816073,5.165305,28.020119,0.004783,0.002674,0.315586
7,745vq78e,5.0,2020-07-03 04:35:30,2020-07-03 11:10:30,1059.240975,4.474331,0.000000,0.000000,0.002425,0.394563
8,745vq78e,5.0,2020-07-21 07:50:00,2020-07-21 13:00:30,1235.004280,4.773593,28.160185,0.064982,0.002675,0.343642
